- crawl 10 phút 1 lần


In [1]:
import requests
import pandas as pd
from datetime import datetime 
import time

In [2]:
with open("./api_key.txt", "r") as f:
    data= f.read()
    API_KEY= data.split("\n")[0]

In [3]:
def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second 
    now = datetime.now()
    timestamp = now.strftime(timestamp_format) 
    with open("./code_log.txt","a",encoding="utf-8") as f: 
        f.write(timestamp + ' : ' + message + '\n')

In [4]:
# Tạo df để lưu tạm
table_attribs = ["dtime","district","Street","speed","speedUncapped","freeFlow","jamFactor","confidence","traversability"]
df = pd.DataFrame(columns= table_attribs)

extend= pd.DataFrame(columns= ["dtime","district","Street","length" ,"speed","speedUncapped","freeFlow","jamFactor","confidence","traversability"])
# DANH SÁCH CÁC KHU VỰC (BBOX) CẦN QUÉT
BBOX_LIST = ["105.8402,21.0192,105.8631,21.0408", 
            "105.8030,21.0225,105.8436,21.0507",
            "105.8043,21.0003,105.8407,21.0289",
            "105.8394,20.9897,105.8698,21.0207",
            "105.8631,21.0188,105.9246,21.0854",
            "105.7234,20.9870,105.7865,21.0365",
            "105.8291,20.9535,105.8817,21.0021",
            "105.7955,21.0464,105.8385,21.0883",
            "105.7350,20.9431,105.8100,21.0045",
            "105.7818,20.9814,105.8320,21.0120",
            "105.7194,21.0372,105.8053,21.0924",
            "105.7610,21.0129,105.8049,21.0452"] 
# Quận Hoàn Kiếm # Quận Ba Đình # Quận Đống Đa # Quận Hai Bà Trưng # Quận Long Biên # Quận Nam Từ Liêm 
# # Quận Hoàng Mai # Quận Tây Hồ # Quận Hà Đông # Quận Thanh Xuân #Bắc Từ Liêm #Cầu Giấy

In [5]:
def extract(bbox: str):
    global df, extend
    url = ""+ bbox + ""+API_KEY
    try:
        response = requests.get(url, timeout=30)  # Thêm timeout để tránh treo
        if response.status_code == 200:
            log_progress(f"Kết nối thành công (status 200) cho bbox: {bbox}")
            json_data= response.json()
            if json_data is None:
                log_progress(f"Không có dữ liệu để load cho bbox: {bbox}")
            else:
                for i in json_data['results']:
                    road_name= i["location"].get("description", "Unknown")
                    lst = [
                        json_data["sourceUpdated"],
                        bbox,
                        road_name,
                        i["currentFlow"]["speed"],
                        i["currentFlow"]["speedUncapped"],
                        i["currentFlow"]["freeFlow"],
                        i["currentFlow"]["jamFactor"],
                        i["currentFlow"]["confidence"],
                        i["currentFlow"]["traversability"]
                    ]
                    df.loc[len(df)]= lst

                    if "subSegments" in i["currentFlow"]:
                            for sub in i["currentFlow"]["subSegments"]:
                                extend[len(extend)] = [
                                    json_data["sourceUpdated"],
                                    bbox,
                                    road_name,
                                    sub.get("length"),
                                    sub.get("speed"),
                                    sub.get("speedUncapped"),
                                    sub.get("freeFlow"),
                                    sub.get("jamFactor"),
                                    sub.get("confidence"),
                                    sub.get("traversability")
                                ]
                
                log_progress(f"Extract thành công cho bbox: {bbox}")
        else:
            log_progress(f"Extract thất bại – status {response.status_code} – {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        log_progress(f"Lỗi mạng khi extract: {e}")
        return None



In [6]:
import os

CSV_FILE = "./hanoi_traffic.csv"

def save_to_csv():
    global df
    try:
        write_header = not os.path.exists(CSV_FILE)
        df.to_csv(
            CSV_FILE,
            mode="a",
            header=write_header,
            index=False,
            encoding="utf-8-sig"
        )
        log_progress(f"Đã lưu {len(df)} dòng dữ liệu vào {CSV_FILE}")
        df = df.iloc[0:0]  # Xóa dữ liệu cũ trong RAM để tránh tràn bộ nhớ
    except Exception as e:
        log_progress(f"Lỗi khi lưu CSV: {e}")


In [7]:
while (True):
    for i in BBOX_LIST:
        log_progress(f"bắt đầu extract: {i}")
        extract(i)
        time.sleep(5)
    save_to_csv()
    log_progress("Hoàn thành 1 lần extract cho HN, chờ 10 phút trước khi lặp lại...")
    time.sleep(600)

KeyboardInterrupt: 

: 